In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import gmaps 
#from api_keys import gkey
import requests
from scipy.stats import linregress

#gmaps.configure(api_key=gkey)

In [12]:
file=pd.read_csv('listings.csv')
file=file.drop(['neighbourhood_group','license','number_of_reviews_ltm','calculated_host_listings_count',
                'host_name','name','host_id','reviews_per_month'],axis=1)
cleaned_data=file.loc[(file['latitude']>= -32.5468)&
                      (file['latitude'] <= -31.7725)&
                      (file['longitude'] <= 116.0593)
                      ]

cleaned_data=cleaned_data.dropna(how='any').sort_values('latitude',ascending=False)
cleaned_data=cleaned_data[(cleaned_data['room_type']=='Entire home/apt')|((cleaned_data['room_type']=='Private room'))]
cleaned_data.drop(cleaned_data[(cleaned_data['neighbourhood']=='PEPPERMINT GROVE')|
                               (cleaned_data['neighbourhood']=='MURRAY')].index, inplace = True)
cleaned_data

,id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,availability_365
5364,38862794,WANNEROO,-31.77287,115.82548,Private room,40,6,18,12/06/2022,348
3423,24808964,WANNEROO,-31.77314,115.87767,Entire home/apt,110,1,178,14/06/2022,330
3070,22778235,JOONDALUP,-31.77332,115.73488,Entire home/apt,128,2,21,27/08/2021,317
3710,27191991,JOONDALUP,-31.77340,115.73292,Entire home/apt,128,2,52,20/05/2022,231
1003,9395509,JOONDALUP,-31.77361,115.73256,Entire home/apt,850,2,12,19/10/2021,231
...,...,...,...,...,...,...,...,...,...,...
4850,34824234,MANDURAH,-32.54627,115.71089,Entire home/apt,600,2,33,12/06/2022,158
1165,10756742,MANDURAH,-32.54628,115.71190,Entire home/apt,995,2,18,27/02/2022,262
8185,52696667,MANDURAH,-32.54635,115.68733,Entire home/apt,186,3,7,16/06/2022,170
8020,52116476,MANDURAH,-32.54639,115.68400,Entire home/apt,523,3,15,16/05/2022,270


In [13]:
#Segregating on room type
private_df = cleaned_data[cleaned_data['room_type'] == 'Private room']
apt_df = cleaned_data[cleaned_data['room_type'] == 'Entire home/apt']


In [14]:
#Segregating based on number of days of stay
#Holiday: 20 days
#short term: 20 to 60 days
#medium term: 60 to 90
#long term: >90 days
cleaned_data = private_df.copy()
bins = [0, 20, 60,90,1000]
group_names = ["Holiday","short term","Medium term","Long Term"]
cleaned_data["Stay classification"] = pd.cut(cleaned_data["minimum_nights"], bins, labels=group_names, include_lowest=True)
cleaned_data

,id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,availability_365,Stay classification
5364,38862794,WANNEROO,-31.77287,115.82548,Private room,40,6,18,12/06/2022,348,Holiday
4169,30162376,SWAN,-31.77423,115.98675,Private room,100,1,2,19/02/2019,342,Holiday
8723,54214407,SWAN,-31.77834,115.99292,Private room,68,3,4,20/05/2022,364,Holiday
4998,35982078,SWAN,-31.78016,115.99217,Private room,42,6,40,22/04/2022,335,Holiday
5151,37272277,SWAN,-31.78034,115.99279,Private room,50,6,16,6/04/2022,139,Holiday
...,...,...,...,...,...,...,...,...,...,...,...
7480,49740161,MANDURAH,-32.52436,115.74105,Private room,85,1,17,12/06/2022,91,Holiday
2194,17672767,MANDURAH,-32.52766,115.76020,Private room,27,1,46,21/12/2019,0,Holiday
6524,45080339,MANDURAH,-32.52845,115.69961,Private room,89,1,38,27/04/2022,302,Holiday
3096,22989423,MANDURAH,-32.53071,115.76852,Private room,55,1,8,1/11/2020,0,Holiday


In [15]:
term_groupby = cleaned_data.groupby(['neighbourhood','Stay classification'])
term_groupby

In [16]:
term_average_price = pd.DataFrame({"Average Price":term_groupby["price"].mean(),
                                 })
term_average_price

Average Price
neighbourhood Stay classification               
ARMADALE      Holiday                  66.153846
              short term                     NaN
              Medium term                    NaN
              Long Term                      NaN
BASSENDEAN    Holiday                  48.571429
...                                          ...
VINCENT       Long Term                      NaN
WANNEROO      Holiday                  49.750000
              short term                     NaN
              Medium term                    NaN
              Long Term                      NaN

[120 rows x 1 columns]

In [17]:
term_average_price.to_csv('avg_price_private_room.csv', encoding='utf-8')               

In [18]:
#Entirehome/ Apt
bins2 = [0, 20, 60,90,1000]
group_names2 = ["Holiday","short term","Medium term","Long Term"]
apt_df["Stay classification"] = pd.cut(apt_df["minimum_nights"], bins2, labels=group_names2, include_lowest=True)
apt_df

C:\Users\bhara\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,availability_365,Stay classification
3423,24808964,WANNEROO,-31.77314,115.87767,Entire home/apt,110,1,178,14/06/2022,330,Holiday
3070,22778235,JOONDALUP,-31.77332,115.73488,Entire home/apt,128,2,21,27/08/2021,317,Holiday
3710,27191991,JOONDALUP,-31.77340,115.73292,Entire home/apt,128,2,52,20/05/2022,231,Holiday
1003,9395509,JOONDALUP,-31.77361,115.73256,Entire home/apt,850,2,12,19/10/2021,231,Holiday
2302,18366020,SWAN,-31.77700,116.05288,Entire home/apt,159,1,91,29/05/2022,80,Holiday
...,...,...,...,...,...,...,...,...,...,...,...
4850,34824234,MANDURAH,-32.54627,115.71089,Entire home/apt,600,2,33,12/06/2022,158,Holiday
1165,10756742,MANDURAH,-32.54628,115.71190,Entire home/apt,995,2,18,27/02/2022,262,Holiday
8185,52696667,MANDURAH,-32.54635,115.68733,Entire home/apt,186,3,7,16/06/2022,170,Holiday
8020,52116476,MANDURAH,-32.54639,115.68400,Entire home/apt,523,3,15,16/05/2022,270,Holiday


In [19]:
term_groupby_2 = apt_df.groupby(['neighbourhood','Stay classification'])
term_groupby_2

In [20]:
term_average_price_2 = pd.DataFrame({"Average Price":term_groupby_2["price"].mean(),
                                 })
term_average_price_2

Average Price
neighbourhood Stay classification               
ARMADALE      Holiday                 172.470588
              short term              160.000000
              Medium term                    NaN
              Long Term                      NaN
BASSENDEAN    Holiday                 132.928571
...                                          ...
VINCENT       Long Term                74.666667
WANNEROO      Holiday                 129.888889
              short term                     NaN
              Medium term                    NaN
              Long Term                      NaN

[120 rows x 1 columns]

In [21]:
term_average_price_2.to_csv('avg_price_entire_apt.csv', encoding='utf-8')    